In [2]:
import pandas as pd

Reading CSV file from S3 into a Data Frame from republished download.bls.gov dataset

In [3]:
csv_df = pd.read_csv(
    f"s3://data-quest-1//pub/time.series/pr/pr.data.0.Current.csv",
    delimiter=r"\s+"
)

csv_df = csv_df[["series_id", "year", "period", "value"]]

csv_df

,series_id,year,period,value
0,PRS30006011,1995,Q01,2.600
1,PRS30006011,1995,Q02,2.100
2,PRS30006011,1995,Q03,0.900
3,PRS30006011,1995,Q04,0.100
4,PRS30006011,1995,Q05,1.400
...,...,...,...,...
36049,PRS88003203,2023,Q03,116.928
36050,PRS88003203,2023,Q04,115.824
36051,PRS88003203,2023,Q05,116.472
36052,PRS88003203,2024,Q01,117.103


Reading JSON file from S3 into a Data Frame from republished datausa.io file

In [5]:
json_df = pd.read_json('s3://data-quest-1/datausa.json')
json_df.columns = json_df.columns.str.lower()

df_filtered_json = json_df.query('2013 <= year <= 2018')

df_filtered_json

,id nation,nation,id year,year,population,slug nation
4,01000US,United States,2018,2018,322903030,united-states
5,01000US,United States,2017,2017,321004407,united-states
6,01000US,United States,2016,2016,318558162,united-states
7,01000US,United States,2015,2015,316515021,united-states
8,01000US,United States,2014,2014,314107084,united-states
9,01000US,United States,2013,2013,311536594,united-states


Using the dataframe from the population data API (datausa.io), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [11]:
print(f"Standard Deviation of population: {round(df_filtered_json['Population'].std(),3)}")

Standard Deviation of population: 4257089.542


In [12]:
print(f"Mean of population: {round(df_filtered_json['Population'].mean())}")

Mean of population: 317437383


Using the dataframe from time series data from download.bls.gov, find the best year for every series_id,: the year with the max/largest sum of "value" for all quarters in that year.

In [6]:
df_best_yr = csv_df.groupby(["series_id", "year"], as_index=False)["value"].sum()

df_best_yr = df_best_yr.loc[df_best_yr.groupby(["series_id"])["value"].idxmax()].reset_index(drop=True)

df_best_yr

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,704.125
3,PRS30006021,2010,17.600
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2023,767.007
279,PRS88003201,2022,36.100
280,PRS88003202,2022,28.900


Generate a report with each series id, the best year for that series, and the summed value for that year.

In [33]:
df_filtered = csv_df.query('series_id == "PRS30006032" and period == "Q01"') 

df_merged = pd.merge(df_filtered, json_df, on=['year'], how='left')[['series_id', 'year', 'period', 'value', 'population']]

df_merged

,series_id,year,period,value,population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.4,NaN
2,PRS30006032,1997,Q01,2.7,NaN
3,PRS30006032,1998,Q01,1.0,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.5,NaN
7,PRS30006032,2002,Q01,-6.7,NaN
8,PRS30006032,2003,Q01,-5.6,NaN
9,PRS30006032,2004,Q01,2.1,NaN
